# COGS 108 - Data Checkpoint

## Authors

*(Replace with your team names and contributions using the [CRediT taxonomy](https://credit.niso.org).)*

- *Andrew Zhang*: Find data and describe dataset 2
- *Andy Cao*: data wrangling for Dataset 1, Writing (original draft)
- *Vicky Huang*: Find data and describe dataset 1
- *Jasmine Lou*: data wrangling for Dataset 2, Writing (original draft)
- *Yiwen Huang*: Update info, Writing (review and editing)

## Research Question

How have housing prices and affordability changed over time in regions surrounding University of California (UC) campuses, and how do these trends relate to local income levels and broader economic conditions?

Specifically, we examine whether housing prices near UC campuses have increased faster than median household income and how these trends differ across regions (e.g., Irvine, La Jolla, Berkeley, Los Angeles) and time periods. The main metrics are median home price (or home value index), median household income, and affordability (e.g., price-to-income ratio or rent burden). The analysis is primarily descriptive and comparative. We will visualize price and income trends, compare regions, and assess changes in affordability over at least 10 to 20 years using public data from government and real-estate sources (e.g., Zillow, U.S. Census/ACS, FRED).



## Background and Prior Work

Housing affordability has become a major economic and social issue in California, particularly in regions with high demand and limited housing supply. Areas surrounding University of California campuses often experience additional housing pressure due to student populations, faculty demand, and local economic growth. Prior research has shown that housing prices in California have increased significantly over the past two decades, frequently outpacing wage growth and contributing to affordability challenges for renters and homeowners alike.

Several public data sources provide context. Zillow's housing market reports document long-term growth in home values across major U.S. metropolitan areas, with especially rapid increases in coastal California cities. U.S. Census data and American Community Survey (ACS) reports provide evidence that median household income growth has been slower and uneven across regions. Our project focuses on UC-adjacent regions (e.g., Irvine, La Jolla, Berkeley, Los Angeles) and compares affordability trends across multiple campuses using Zillow, Census/ACS, and FRED data.

## Hypothesis


We hypothesize that housing prices in regions surrounding UC campuses have increased faster than median household income over time, leading to decreased affordability. We also expect that UC regions in major metropolitan or coastal areas (e.g., near UC Berkeley or UCLA) will exhibit higher prices and lower affordability compared to UC campuses in less dense regions (e.g., UC Riverside or UC Merced). This is based on prior evidence that coastal and urban California markets have seen stronger price growth and that university towns often face extra demand from students and staff, while income growth has been slower and uneven across the state.


## Data

### Data overview

**Dataset 1: Zillow Home Value Index (ZHVI), Single-Family Residences**
- **Dataset name:** Zip_Zhvi_SingleFamilyResidence (ZIP code level, single-family home values).
- **Link:** [Zillow Research Data](https://www.zillow.com/research/data/). For this checkpoint we use a public mirror: `https://raw.githubusercontent.com/noahgift/real_estate_ml/master/data/Zip_Zhvi_SingleFamilyResidence_2018.csv`.
- **Observations:** One row per ZIP code. Thousands of ZIPs nationally; we filter to California and further to UC-adjacent areas.
- **Variables:** RegionID, RegionName (ZIP), State, Metro, CountyName, and many date columns (e.g., 1996-04 to 2018-08) with typical home value in dollars for that month. We use RegionName/State/CountyName for geography and the date columns for price levels and trends over time.
- **Relevance:** Provides housing price levels and growth over time at fine geography (ZIP). We can aggregate to county or metro and align with UC campus regions (e.g., Irvine, La Jolla, Berkeley, Los Angeles).
- **Shortcomings:** Data end in 2018 in this mirror. Official Zillow data are updated monthly. Geography is ZIP, so we must map ZIPs to counties/metros for UC regions. Single-family only (excludes condos).

**Dataset 2: American Community Survey (ACS) – Median Household Income**

- **Dataset name:** ACS 5-Year Estimates, Table B19013
- **Link:** https://data.census.gov/table/ACSDT5Y2023.B19013?q=ACS+5-year+B19013+county+California&g=040XX00US06$0500000
- **Source:** U.S. Census Bureau  
- **Observations:** One row per county per year  

- **Variables:** County name, year, median household income (USD)  

- **Relevance:** Provides multi-year county-level income data, enabling comparison of income growth with housing price growth over time.  

- **Shortcomings:** ACS is survey-based and represents estimates rather than exact administrative records.

**Combining the datasets:** We will join on geography. Zillow data have CountyName; income data have county. We will filter Zillow to California, aggregate or select ZIPs for UC-adjacent counties (e.g., Orange, San Diego, Alameda, Los Angeles), and merge with county-level income. For time series we use Zillow date columns and, when available, multi-year income data (e.g., from Census ACS) for the same counties.


In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import sys
sys.path.append('./modules')

import get_data

datafiles = [
    {'url': 'https://raw.githubusercontent.com/noahgift/real_estate_ml/master/data/Zip_Zhvi_SingleFamilyResidence_2018.csv', 'filename': 'Zip_Zhvi_SingleFamilyResidence_2018.csv'},
    {'url': 'https://data.ca.gov/dataset/d56fc70f-5566-4030-8854-1ce72c93e100/resource/49eb1f40-d50a-4dde-98ca-0450d69c4617/download/2022-income-limits.csv', 'filename': '2022-income-limits.csv'}
]

get_data.get_raw(datafiles, destination_directory='data/00-raw/')

Overall Download Progress:  50%|█████     | 1/2 [00:00<00:00,  1.45it/s]

Successfully downloaded: Zip_Zhvi_SingleFamilyResidence_2018.csv


Overall Download Progress: 100%|██████████| 2/2 [00:01<00:00,  1.37it/s]

Successfully downloaded: 2022-income-limits.csv


### Zillow Home Value Index (ZHVI), Single-Family by ZIP

**Metrics and units.** The dataset gives the Zillow Home Value Index (ZHVI) for single-family residences at the ZIP code level. Each date column (e.g., 1996-04, 2018-08) contains the estimated typical home value in **dollars (USD)** for that month. ZHVI aims to represent the “middle” of the housing stock (e.g., 35th–65th percentile). Values in the hundreds of thousands are normal for many California ZIPs; very low values (e.g., under 50,000) or extremely high ones may indicate data issues or unusual markets. Geography is given by RegionID, RegionName (ZIP code), State, Metro, and CountyName.

**Concerns.** (1) This mirror ends in 2018; for newer years we would use official Zillow Research data. (2) Coverage can be better in active listing markets, so some rural or low-transaction ZIPs may be missing or noisier. (3) Single-family only, so condos and multi-family are excluded. (4) We filter to California and UC-adjacent counties, so national coverage is not used in the analysis.


In [7]:
import pandas as pd
import os

path_raw = 'data/00-raw/Zip_Zhvi_SingleFamilyResidence_2018.csv'
zhvi_raw = pd.read_csv(path_raw)

print('Shape:', zhvi_raw.shape)
print('Columns (first 10 and last 5):', list(zhvi_raw.columns[:10]) + ['...'] + list(zhvi_raw.columns[-5:]))

zhvi_ca = zhvi_raw[zhvi_raw['State'] == 'CA'].copy()
print('\nRows (CA only):', len(zhvi_ca))

key_cols = ['RegionID', 'RegionName', 'State', 'CountyName', 'Metro']
date_cols = [c for c in zhvi_ca.columns if c not in key_cols]
print('\nMissing in key cols:')
print(zhvi_ca[key_cols].isna().sum())
print('Sample missing % (first date col):', zhvi_ca[date_cols[0]].isna().mean().round(4))
print('Sample missing % (last date col):', zhvi_ca[date_cols[-1]].isna().mean().round(4))

date_vals = zhvi_ca[date_cols]
zhvi_ca = zhvi_ca[(date_vals.notna().any(axis=1)) & (date_vals != 0).any(axis=1)]
print('\nRows after dropping all-NaN/all-zero date rows:', len(zhvi_ca))

os.makedirs('data/02-processed', exist_ok=True)
zhvi_ca.to_csv('data/02-processed/zhvi_sfr_zip_ca_2018.csv', index=False)
print('\nSaved cleaned CA ZHVI to data/02-processed/zhvi_sfr_zip_ca_2018.csv')

last_col = date_cols[-1]
print('\nSummary of', last_col, '(CA ZIPs):')
print(zhvi_ca[last_col].describe())


Shape: (15508, 280)
Columns (first 10 and last 5): ['RegionID', 'RegionName', 'City', 'State', 'Metro', 'CountyName', 'SizeRank', '1996-04', '1996-05', '1996-06', '...', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12']

Rows (CA only): 1229

Missing in key cols:
RegionID       0
RegionName     0
State          0
CountyName     0
Metro         45
dtype: int64
Sample missing % (first date col): 0.0
Sample missing % (last date col): 0.0

Rows after dropping all-NaN/all-zero date rows: 1229

Saved cleaned CA ZHVI to data/02-processed/zhvi_sfr_zip_ca_2018.csv

Summary of 2018-12 (CA ZIPs):
count    1.229000e+03
mean     7.485329e+05
std      6.628661e+05
min      1.171000e+05
25%      3.533000e+05
50%      5.685000e+05
75%      8.854000e+05
max      6.671700e+06
Name: 2018-12, dtype: float64


### American Community Survey (ACS) — Median Household Income

**Metrics and units.**  
This dataset comes from the American Community Survey (ACS) 5-Year Estimates, Table B19013, published by the U.S. Census Bureau. It contains county-level estimates of median household income measured in **U.S. dollars (USD)**. Each row represents a county observation, and the key variable is median household income, which represents the midpoint of household income distribution within a county. This means half of households earn more than the median income and half earn less.

**Relevance.**  
Median household income is an important economic indicator because it reflects the general income level and economic well-being of residents in a geographic region. Higher median income values typically indicate greater economic resources available to households, while lower values may reflect economic disadvantage or limited employment opportunities. Because the ACS provides standardized estimates across counties, this dataset allows comparison of income levels across regions and over time.

**Concerns.**  
ACS values are estimates derived from survey sampling rather than complete administrative records. As a result, there may be sampling error or uncertainty in the reported values. Additionally, median household income does not capture income inequality within a county and does not account for differences in cost of living across regions.


In [8]:
import pandas as pd
import os

# Use the second dataset actually downloaded in Cell 12: 2022-income-limits.csv (CA county AMI)
data_path = "data/00-raw/2022-income-limits.csv"
df = pd.read_csv(data_path)

print("Original columns (first 5):", list(df.columns[:5]))
print("Original shape:", df.shape)

# County-level Area Median Income (AMI) is a standard proxy for median household income
df = df.rename(columns={
    "County": "county",
    "AMI": "median_household_income"
})

# Add year to match description: one row per county per year (this file is 2022)
df["year"] = 2022
df = df[["county", "year", "median_household_income"]]

df["median_household_income"] = pd.to_numeric(
    df["median_household_income"],
    errors="coerce"
)
print("\nMissing values:")
print(df.isna().sum())

df = df.dropna()

print("\nCleaned dataset shape:", df.shape)
print("\nPreview of cleaned dataset:")
print(df.head())

print("\nSummary statistics:")
print(df["median_household_income"].describe())

os.makedirs("data/02-processed", exist_ok=True)
df.to_csv("data/02-processed/income_limits_ca_2022.csv", index=False)
print("\nSaved to data/02-processed/income_limits_ca_2022.csv")

Original columns (first 5): ['County', 'AMI', 'ALI_1', 'ALI_2', 'ALI_3']
Original shape: (58, 42)

Missing values:
county                     0
year                       0
median_household_income    0
dtype: int64

Cleaned dataset shape: (58, 3)

Preview of cleaned dataset:
      county  year  median_household_income
0    Alameda  2022                   142800
1     Alpine  2022                    94900
2     Amador  2022                    86600
3      Butte  2022                    85000
4  Calaveras  2022                    90000

Summary statistics:
count        58.000000
mean      97486.206897
std       24447.576913
min       80300.000000
25%       80300.000000
50%       87000.000000
75%      106225.000000
max      168500.000000
Name: median_household_income, dtype: float64

Saved to data/02-processed/income_limits_ca_2022.csv


##Information Updates


Since the project proposal, we refined our research focus and improved several parts of the project design. First, we clarified the research question to more specifically examine how housing prices have changed relative to median household income in UC-adjacent regions over time. We also made the hypothesis more measurable by specifying expected differences in price growth and affordability between coastal and inland UC regions.

In terms of data, we confirmed and obtained the Zillow Home Value Index dataset and ACS median household income data. We cleaned the datasets, examined missing values, and verified that they can be merged at the county level for analysis.

We updated our project timeline to reflect completed steps, including dataset acquisition and preprocessing, and to outline more realistic next steps for exploratory analysis and statistical modeling.

Finally, we expanded our ethics considerations by addressing potential dataset bias, survey limitations, and how our interpretation of housing affordability trends may impact public understanding of economic inequality.


## Ethics

We use only aggregated, publicly available data (Zillow, Census/ACS, CA Open Data). There are no human subjects. We considered collection bias (Zillow may overrepresent urban/higher-income markets; we state this and examine regional differences), PII (none collected), and dataset bias (we avoid normative claims and frame results as regional trends). Data are stored in the repo with normal access; no model is deployed. See the full Ethics checklist in 00-ProjectProposal.ipynb for all items.

## Team Expectations

All team members have read the [COGS108 Team Policies](https://github.com/COGS108/Projects/blob/master/COGS108_TeamPolicies.md) and agree to the following expectations.

* **Communication.** Communicate regularly through a shared platform (e.g., Discord or Slack) and attend scheduled meetings.
* **Contribution.** Each member is expected to contribute equitably to data collection, analysis, and writing.
* **Conflict resolution.** If conflicts arise, we agree to address them respectfully and promptly through group discussion.
* **Commitment.** By submitting this proposal, each member affirms that they have read the COGS 108 Team Policies and intend to meet these expectations.


## Project Timeline Proposal

We do not anticipate needing specialized methods beyond those covered in COGS 108. Standard data analysis, visualization, and basic statistical techniques should be sufficient.

| Week | Completed Before | Discuss / Deliverables |
|------|------------------|-------------------------|
| 3 to 4 | Finalize research question and identify datasets (Zillow, Census/ACS, FRED) | Confirm data access and align on UC regions (e.g., Irvine, La Jolla, Berkeley, LA) |
| 5 | Data cleaning and preprocessing | Joined region-time tables, consistent geography and time range |
| 6 | Exploratory data analysis and visualization | Price and income trends, price difference and income comparison visuals |
| 7 | Statistical analysis and interpretation | Compare price vs. income growth, affordability by region |
| 8 | Draft results and ethics discussion | Integrate limitations and ethics, first full draft |
| 9 | Finalize analysis and visualizations | Polished figures and tables, finalize methods and results |
| 10 | Complete final report and presentation | Turn in final project and group surveys |

**Update for Data Checkpoint:** We obtained Zillow ZHVI (ZIP, single-family) and CA county income limits. Next we will add Census ACS median household income by county/year for time trends and refine the county list for UC-adjacent regions before EDA.